# Processing lyrics database

### Database
https://www.kaggle.com/datasets/jcecheverria/bad-bunny-lyrics



In [1]:
!pip3 install -U scikit-learn
import pandas as pd
import re
from sklearn.manifold import TSNE
import spacy


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
bb_db = pd.read_csv("../data/bad_bunny_lyrics.csv")
print(bb_db.head())
lyrics_list = bb_db['lyrics']
lyrics_es2coro = [l.replace("Estribillo", "Coro") for l in lyrics_list]
bb_db['lyrics'] = lyrics_es2coro

     artists     album               title  title_with_featured  \
0  Bad Bunny  X 100pre      NI BIEN NI MAL       NI BIEN NI MAL   
1  Bad Bunny  X 100pre             200 MPH  200 MPH (Ft. Diplo)   
2  Bad Bunny  X 100pre     ¿Quién Tú Eres?      ¿Quién Tú Eres?   
3  Bad Bunny  X 100pre                Caro                 Caro   
4  Bad Bunny  X 100pre  Tenemos Que Hablar   Tenemos Que Hablar   

                                              lyrics  \
0  [Letra de "NI BIEN NI MAL"]\n\n[Intro: Bad Bun...   
1  [Letra de "200 MPH" ft. Diplo]\n\n[Intro]\n¡Ju...   
2  [Letra de "¿Quién Tú Eres?"]\n\n[Intro]\nUh-uh...   
3  [Letra de "Caro"]\n\n[Estribillo: Bad Bunny]\n...   
4  [Letra de "Tenemos Que Hablar"]\n\n[Intro]\nYe...   

                                                 url  
0  https://genius.com/Bad-bunny-ni-bien-ni-mal-ly...  
1        https://genius.com/Bad-bunny-200-mph-lyrics  
2  https://genius.com/Bad-bunny-quien-tu-eres-lyrics  
3           https://genius.com/Bad-bunny

In [3]:
bb_db['lyrics'][2]

'[Letra de "¿Quién Tú Eres?"]\n\n[Intro]\nUh-uh, uh-uh-uh-uh\nRrrrr, ah-ah, ah-ah-ah-ah\nRrrrr, uh-uh, uh-uh-uh-uh\n¡Rrrrrrah!\n\n[Coro]\n¿Quién tú eres? Ey\nDime, socio, ¿quién tú ere\'?\nPa\' frontearme a mí, ¿quién tú eres? Ey\nHabla claro, ¿quién tú ere\'? Ey\nHuele bicho, ¿quién tú ere\'? Ey\nDime, socio, ¿quién tú ere\'?\nPa\' frontearme a mí, ¿quién tú eres? Ey\nHabla claro, ¿quién tú eres?\nYeah, ven acá\n\n[Verso]\n¿Cabrón, tú me pagas los biles? (¡No!)\nTú hablando de mí y yo haciendo mile\'\nOtro palo pa\' encima \'e los files\', ey\nTú robando en Macy\'s y yo en el desfile\nHoy quiero una uruguaya y otra de Chile, ey, ey\nOtro que me tira (¿Ah?)\nCabrone\', entiendan que pa\' acá no se mira (¡No!)\nYa mismo te cae, te tengo en la mira (Yeh; prr)\nTú metío\' en tu casa y yo de gira (Jaja)\nVoy a las millas, como Toretto (¡Fum!)\n¿Qué tú ere\' el más duro? Te reto\nYo tengo el imán, yo soy el Magneto, ey\nA tu envidia en la cara le meto (Tss, tss-tss)\nIván Calderón, pero a p

In [4]:
def strip_lyric_header(song_lyrics):
    """remove the header with artist and song name"""
    start = 0
    for ix, character in enumerate(list(song_lyrics)):
        if character == "]":
            start = ix
            break
    return song_lyrics[start+1:]

def format_lyrics_string(l_string):
    """ """
    # fmt_str = l_string.replace("\n", " ").replace("[", "").replace("]", "")
    fmt_str = l_string.replace("[", "").replace("]", "")
    fmt_str = re.sub(r'(\w)([.,?!;:])', r'\1 \2', fmt_str)
    return fmt_str

def get_section_starts(lyrics_nh):
    """ """
    targets = ["Coro", "Intro", "Verso", "Outro"]
    targets_index = {t: ix for ix, t in enumerate(targets)}
    keys = ['intro',
            'verse',
            'chorus',
            'outro']
    
    section_ix = []
    section_type = []
    
    # find all section starts. store the starting index and categorize.
    for ix, char in enumerate(list(lyrics_nh)):
        if char == "[":
            for t in targets:
                if t in lyrics_nh[ix:ix+10]:
                    for ix_end, char_end in enumerate(lyrics_nh[ix:]):
                        if char_end == "]": 
                            section_ix.append((ix, ix+ix_end))
                            section_type.append(t)
                            break  
    return section_type, section_ix
    
def make_body_ix(section_ix, lyrics_len):
    """ """
    sections = []
    for i, sec in enumerate(section_ix):
        if sec == section_ix[-1]:
            sections.append((sec[-1], lyrics_len))
        else:
            sections.append((sec[-1], section_ix[i+1][0]))    
           
    return sections
    
def make_lyrics_dict(lyrics, title):
    """ """
    lyrics_dict = {key: [] for key in ["Coro", "Intro", "Verso", "Outro"]}
    lyrics_len = len(lyrics)
    section_type, section_ix = get_section_starts(lyrics)
    sections = make_body_ix(section_ix, lyrics_len)
    for t, sec in zip(*(section_type, sections)):
        lyrics_dict[t].append(lyrics[sec[0]:sec[1]]) #add processing later
    return lyrics_dict

def remove_section_headers(lyrics):
    """"""
    new_str_list = []
    sw = 0
    for i, ch  in enumerate(list(lyrics)):

        if ch == "[":
            sw = 1
        elif ch == "]":
            sw = 0
        else:
            if not sw:
                new_str_list.append(ch)
            else:
                pass
    return ''.join(new_str_list)
    


In [25]:
lyrics = bb_db['lyrics'][4]
lyrics_len = len(lyrics)
d = make_lyrics_dict(lyrics, 'TITLE')
# print(get_section_starts(lyrics))
# section_type, section_ix = get_section_starts(lyrics)
# print(make_body_ix([(33, 39), (97, 102), (360, 368), (971, 976), (1371, 1379), (1647, 1652)], lyrics_len))

In [36]:
s = re.sub(r'(\w)([.,?!;:])', r'\1 \2', d["Coro"][0])
s.replace("\n", " ").replace("[", "").replace("]", "").replace("¿", "").replace("?", "")


' Odio tus mensajes , cuando dices Que tenemos que hablar Oh Dios , qué yo hice , qué yo hice  Que tenemos que hablar Odio tus mensajes , cuando dices Que tenemos que hablar Oh Dios , qué yo hice , qué yo hice  Que tenemos que hablar Yeah , yeah , yeah , yeah  '

In [7]:
list("23kskjfjls")

['2', '3', 'k', 's', 'k', 'j', 'f', 'j', 'l', 's']

In [42]:
bb_db['lyrics_no_headers'] = [format_lyrics_string(remove_section_headers(l)).replace('\\', '').replace("'", "").replace("\xa1", "") for l in bb_db['lyrics']]
bb_db['lyrics_no_headers'][36].replace("\u2005", " ")

'   Bla , bla , bla , bla , bla , bla Ey , yo, yo, yo Yo, yo, yo, yo, yah Lalalalalalala (Blow , blow) Lalalalalalala   Diablo , qué safaera Tú tiene un culo cabrón Cualquier cosa que te ponga rompe la carretera (Lalalalala ; aight) Muévelo , muévelo , muévelo , muévelo (Lalalalalalala) Qué safaera\u205f(Lalalalala) Tiene\u205fun\u205fculo cabrón Cualquier cosa\u205fque te ponga\u205frompe la carretera (Aight ; tra !) Muévelo , muévelo , muévelo , muévelo   Qué falta de respeto , mami ¿Cómo te atreve a venir sin panty ? Hoy saliste puesta pa mí Yo que pensaba que venía a dormir , no Vino ready ya , puesta pa una cepillá Me chupa la lollipop , solita se arrodilla , hey ¿Cómo te atreve, mami , a venir sin panty ?   Mera , dímelo , DJ Orma ¿Qué tú te cree? Jodío cabrón , jeje Yo Hago Lo Que Me Da La Gana Díselo , Conejo Ey , ey (Jajajaja)   Hoy se bebe , hoy se gasta Hoy se fuma como un rasta Si Dio lo permite (Si Dio lo permite), ey Si Dio lo permite (Que si Dio lo permite), ey Hoy se be

In [21]:
bb_db.to_csv("/Users/jrd/Documents/DM2023/bb_db.csv", encoding='latin-1')


UnicodeEncodeError: 'latin-1' codec can't encode character '\u2014' in position 1302: ordinal not in range(256)

In [23]:
import transformers
from transformers import BloomForCausalLM
from transformers import BloomTokenizerFast
import torch

In [24]:
model = BloomForCausalLM.from_pretrained("bigscience/bloom-1b3")
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloom-1b3")

OSError: bigscience/bloom-1b3 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [27]:
import os

In [ ]:
os.system()